In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist, FieldError
file = "archiv/data/Database_Tyrol_InProgress.xlsx"
df = pd.read_excel(file, "MONUMENT PROTECTION")[3:].fillna('False')

In [ ]:
failed_sites = []
for i, row in df.iterrows():
    try:
        temp_site = Site.objects.get(identifier=row['*SiteID'])
    except ObjectDoesNotExist:
        failed_sites.append(row['*SiteID'])
        temp_site = None
    if temp_site:
        
        if row['*Monument Protection ID'] != 'False':
            temp_monument, _ = MonumentProtection.objects.get_or_create(
                identifier=row['*Monument Protection ID']
            )
        else:
            temp_monument, _ = MonumentProtection.objects.get_or_create(
                identifier="{}.{}".format(row['*SiteID'], i)
            )
        temp_monument.site_id = temp_site
        
        try:
            if row['Public'] == "yes":
                temp_monument.public = True
        except KeyError:
            pass
            
       
        if row['Threats'] != 'False':
            item = None
            x = None
            for x in row['Threats'].split(','):
                x = x.strip()
                try:
                    item = SkosConcept.objects.get(pref_label=x)
                except ObjectDoesNotExist:
                    item = None
                if item:
                    temp_monument.threats.add(item)
        
        if row['Heritage Status'] != 'False':
            if row['Heritage Status'].endswith('no'):
                temp_monument.heritage_status = False
        
        landuse = ";".join(
            [
                row['agricultural areas'],
                row['artificial surfaces'],
                row['forests and semi-natural areas'],
                row['water bodies'],
                row['wetlands'],

            ]
        )
        for x in landuse.split(';'):
            item = None
            if x != 'False':
                x = x.strip()
                try:
                    item = SkosConcept.objects.get(pref_label=x)
                except ObjectDoesNotExist:
                    item = None
                if item:
                    temp_monument.current_land_use.add(item)

        temp_monument.save()
    

In [ ]:
# MonumentProtection.objects.all().delete()